# Data Requirements and Setup Instructions

This notebook requires the following data files to be placed in appropriate directories:

## Required Data Sources:

1. **O*NET Database Files** (place in `{basepath}/onet_data/`):
   - `Skills Nov 2019.xlsx` - Download from O*NET Database at https://www.onetcenter.org/database.html
   - `Abilities Nov 2019.xlsx` - From same O*NET Database
   - `Knowledge Nov 2019.xlsx` - From same O*NET Database  
   - `Occupation Data.txt` - O*NET occupation codes and titles

2. **BLS Wage Data** (place in `{basepath}/bls_wage_data/`):
   - `national_M2019_dl.xlsx` - Download from Bureau of Labor Statistics OES data at https://www.bls.gov/oes/

3. **Historical O*NET Data** (place in `{basepath}/onet_historical/`):
   - `EducTrainExp.9.txt` - O*NET version 5.0 education data for historical comparison

4. **Processed Data Files** (create these or place in `{basepath}/processed_data/`):
   - `binary_skill_occupation_bipartite_serrano_filtered.csv` - Binary bipartite network created using Serrano et al (2009) method
   - `skill_dependency_data_baldwin_method.csv` - Skill dependency data computed using Baldwin (2010) method

5. **Custom Data Files** (create based on your analysis):
   - `skill_clustering_data.csv` - Skill categorizations (General/Intermediate/Specific)
   - `occupational_education_data.csv` - Education requirements by occupation

## Directory Structure:
```
{basepath}/
├── onet_data/
├── bls_wage_data/ 
├── onet_historical/
├── processed_data/
├── output_data/
│   ├── figures/
│   │   └── sensitivity_analysis/
│   └── gephi_files/
└── working_directory/
    └── skill_capital_wage_distribution/
```

Set the `basepath` variable in the first code cell to point to your data directory.

# Loading Packages

In [ ]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import distanceclosure as dc
import copy
from collections import ChainMap
from scipy import integrate
from matplotlib import pyplot as plt
from tqdm import tqdm
# from umap import UMAP # requires numpy version < 1.20.0
# from sklearn.model_selection import train_test_split




# from dfply import *

## Changing Working Directory
print(os.getcwd())
# Set basepath to your main data directory
# Replace with the path to your data directory containing the required datasets
basepath = "/path/to/your/data/directory/"

# Change to working directory containing skill-capital-wage distribution data
os.chdir(basepath+"working_directory/skill_capital_wage_distribution")
print(os.listdir)


/Users/mohhosseinioun/Library/CloudStorage/Dropbox/University/PhD/2020/Research/Skill - Capital - Wage Distribution/Code/Network Prep and Analysis
<built-in function listdir>
['230.0000000000001, 7.0000000000000355, 41.99999999999996', '89, 89, 89', '66.00000000000007, 78.99999999999989, 164.00000000000006']


# Forming Skill Data

In [ ]:
## Loading Data
# Load O*NET Skills data - download from O*NET Database at https://www.onetcenter.org/database.html
skill_occ = pd.read_excel(basepath+"onet_data/"+"Skills Nov 2019.xlsx")

# Load O*NET Abilities data
ability_occ = pd.read_excel(basepath+"onet_data/"+"Abilities Nov 2019.xlsx")

# Load O*NET Knowledge data  
know_occ = pd.read_excel(basepath+"onet_data/"+"Knowledge Nov 2019.xlsx")

skill_occ.loc[skill_occ['Element Name'] == "Mathematics", 'Element Name'] = "Mathematics Skills"
know_occ.loc[know_occ['Element Name'] == "Mathematics", 'Element Name'] = "Mathematics Knowledge"

print(skill_occ.head())
print(ability_occ.head())
print(know_occ.head())

  O*NET-SOC Code             Title Element ID           Element Name Scale ID  \
0     11-1011.00  Chief Executives    2.A.1.a  Reading Comprehension       IM   
1     11-1011.00  Chief Executives    2.A.1.a  Reading Comprehension       LV   
2     11-1011.00  Chief Executives    2.A.1.b       Active Listening       IM   
3     11-1011.00  Chief Executives    2.A.1.b       Active Listening       LV   
4     11-1011.00  Chief Executives    2.A.1.c                Writing       IM   

   Scale Name  Data Value    N  Standard Error  Lower CI Bound  \
0  Importance        4.12  8.0            0.13            3.88   
1       Level        4.75  8.0            0.16            4.43   
2  Importance        4.12  8.0            0.13            3.88   
3       Level        4.88  8.0            0.23            4.43   
4  Importance        4.00  8.0            0.00            4.00   

   Upper CI Bound Recommend Suppress Not Relevant       Date Domain Source  
0            4.37                  N   

In [ ]:
## Combining Skills
### Adding Type
skill_occ['type'] = "skill"
ability_occ['type'] = "ability"
know_occ['type'] = "knowledge"

# ### Renaming Columns
skill_occ = skill_occ.rename(columns = {'O*NET-SOC Code':'occ_8_dig', 'Element ID':'element_ID', 'Element Name':'element_title'})
ability_occ = ability_occ.rename(columns = {'O*NET-SOC Code':'occ_8_dig', 'Element ID':'element_ID', 'Element Name':'element_title'})
know_occ = know_occ.rename(columns = {'O*NET-SOC Code':'occ_8_dig', 'Element ID':'element_ID', 'Element Name':'element_title'})


### Creating Skills' Names
skills_names = pd.concat([skill_occ[['element_ID', 'element_title', 'type']],
                          ability_occ[['element_ID', 'element_title', 'type']],
                          know_occ[['element_ID', 'element_title', 'type']]], 
                         ignore_index=True).drop_duplicates()

# Load occupation names and codes - available from O*NET Database
occ_names = pd.read_csv(basepath+"onet_data/"+'Occupation Data.txt', sep = "\t") \
    .rename(columns = {'O*NET-SOC Code':'occ_8_dig'})[['occ_8_dig','Title']]


### Keeping Needed Columns and Pivoting Values
skill_occ = skill_occ[['occ_8_dig', 'element_ID', 'Scale ID', 'Data Value', 'type']] \
    .pivot_table(index = ['occ_8_dig', 'element_ID'], columns = 'Scale ID', values = 'Data Value').reset_index(drop = False)
ability_occ = ability_occ[['occ_8_dig', 'element_ID', 'Scale ID', 'Data Value', 'type']] \
    .pivot_table(index = ['occ_8_dig', 'element_ID'], columns = 'Scale ID', values = 'Data Value').reset_index(drop = False)
know_occ = know_occ[['occ_8_dig', 'element_ID', 'Scale ID', 'Data Value', 'type']] \
    .pivot_table(index = ['occ_8_dig', 'element_ID'], columns = 'Scale ID', values = 'Data Value').reset_index(drop = False)

### Combining Skills
skills_occ_raw = pd.concat([skill_occ.rename(columns={'IM': 'Importance', 'LV': 'Level'}),
                            ability_occ.rename(columns={'IM': 'Importance', 'LV': 'Level'}),
                            know_occ.rename(columns={'IM': 'Importance', 'LV': 'Level'})],
                           ignore_index=True)[["occ_8_dig", "element_ID", "Importance", "Level"]]


print(skills_occ_raw.head())
print(skills_names.head())

Scale ID   occ_8_dig element_ID  Importance  Level
0         11-1011.00    2.A.1.a        4.12   4.75
1         11-1011.00    2.A.1.b        4.12   4.88
2         11-1011.00    2.A.1.c        4.00   4.38
3         11-1011.00    2.A.1.d        4.38   4.88
4         11-1011.00    2.A.1.e        3.25   3.62
  element_ID          element_title   type
0    2.A.1.a  Reading Comprehension  skill
2    2.A.1.b       Active Listening  skill
4    2.A.1.c                Writing  skill
6    2.A.1.d               Speaking  skill
8    2.A.1.e     Mathematics Skills  skill


In [ ]:
## Loading Other Attributes
# Load skill clustering data - create this file with skill categorizations (General/Intermediate/Specific)
skill_clusters = pd.read_csv(basepath+"skill_clustering_data.csv")

skill_clusters['skill_Cluster'] = skill_clusters['skill_Cluster']. \
    astype(pd.CategoricalDtype(categories=["General", "Intermediate", "Specific"], ordered=True))
skill_clusters['gen_related'] = skill_clusters['gen_related']. \
    astype(pd.CategoricalDtype(categories=["General", "Nested Intermediate", "Nested Specific", "Un-nested Intermediate", "Un-nested Specific"], ordered=True))

# Load occupational education data - create from O*NET Education, Training, and Experience data
occ_educ = pd.read_csv(basepath+"occupational_education_data.csv")
skill_educ = occ_educ[['occ_8_dig' ,'Education.Avg']] \
    .merge(skills_occ_raw[["occ_8_dig", "element_ID", "Importance"]], on = "occ_8_dig") \
    .groupby(['element_ID']).apply(lambda g: round(np.average(g['Education.Avg'], weights=g['Importance']), 3)) \
    .rename('Imp_weight_Avg_Edu').reset_index(drop = False) \
    .merge(occ_educ[['occ_8_dig' ,'Education.Avg']] \
        .merge(skills_occ_raw[["occ_8_dig", "element_ID", "Level"]], on = "occ_8_dig") \
        .groupby(['element_ID']).apply(lambda g: round(np.average(g['Education.Avg'], weights=g['Level']), 3)) \
        .rename('Lvl_weight_Avg_Edu').reset_index(drop = False))




print(skill_clusters.head())
print(skill_educ.head())

   Unnamed: 0 element_ID skill_Cluster          gen_related
0           1  1.A.1.a.1       General              General
1           2  1.A.1.a.2       General              General
2           3  1.A.1.a.3       General              General
3           4  1.A.1.a.4       General              General
4           5  1.A.1.b.1  Intermediate  Nested Intermediate
  element_ID  Imp_weight_Avg_Edu  Lvl_weight_Avg_Edu
0  1.A.1.a.1               4.877               5.036
1  1.A.1.a.2               5.001               5.158
2  1.A.1.a.3               4.911               5.059
3  1.A.1.a.4               5.056               5.261
4  1.A.1.b.1               5.003               5.168


In [ ]:
# Load BLS Occupational Employment Statistics wage data - download from https://www.bls.gov/oes/
occ_wages = pd.read_excel(basepath + "bls_wage_data/national_M2019_dl.xlsx")[['occ_code', 'a_mean']]
occ_wages.a_mean[occ_wages.a_mean == "#"] = 208001
occ_wages.a_mean[occ_wages.a_mean == "*"] = np.nan
occ_wages.a_mean[occ_wages.a_mean == "**"] = np.nan
occ_wages.a_mean = occ_wages.a_mean.astype(float)

print(occ_wages.head())

skills_occ_raw_temp = skills_occ_raw[["occ_8_dig", "element_ID", "Level"]]
skills_occ_raw_temp["occ_code"] = skills_occ_raw_temp["occ_8_dig"].str[:7]
occ_wages['log_wage'] = np.log10(occ_wages['a_mean'])

skill_wages = occ_wages[["occ_code", "log_wage"]] \
    .merge(skills_occ_raw_temp[["occ_code", "element_ID", "Level"]], on = "occ_code").dropna() \
    .groupby(['element_ID']).apply(lambda g: round(np.average(g['log_wage'], weights=g['Level']), 3)) \
    .rename('log_wage').reset_index(drop = False)

print(skill_wages.head())

  occ_code    a_mean
0  00-0000   53490.0
1  11-0000  122480.0
2  11-1000  127070.0
3  11-1010  193850.0
4  11-1011  193850.0
  element_ID  log_wage
0  1.A.1.a.1     4.785
1  1.A.1.a.2     4.793
2  1.A.1.a.3     4.786
3  1.A.1.a.4     4.801
4  1.A.1.b.1     4.798


In [9]:
skills_occ_raw_temp = skills_occ_raw[["occ_8_dig", "element_ID", "Level"]]
skills_occ_raw_temp["occ_code"] = skills_occ_raw_temp["occ_8_dig"].str[:6]

print(skills_occ_raw_temp)

Scale ID   occ_8_dig element_ID  Level occ_code
0         11-1011.00    2.A.1.a   4.75   11-101
1         11-1011.00    2.A.1.b   4.88   11-101
2         11-1011.00    2.A.1.c   4.38   11-101
3         11-1011.00    2.A.1.d   4.88   11-101
4         11-1011.00    2.A.1.e   3.62   11-101
...              ...        ...    ...      ...
116155    53-7121.00    2.C.7.e   0.33   53-712
116156    53-7121.00    2.C.8.a   2.61   53-712
116157    53-7121.00    2.C.8.b   1.52   53-712
116158    53-7121.00    2.C.9.a   0.93   53-712
116159    53-7121.00    2.C.9.b   1.23   53-712

[116160 rows x 4 columns]


# Creating Bipartite Network

## Filtering Out Non-essential Links and Transforming to Binary Bipartite

In [ ]:
## Approach 1: Using Serrano et al method for binary network creation
# Load pre-computed binary bipartite network - see companion notebook for creation process
# This file contains filtered occupation-skill connections using Serrano et al (2009) method
bipartite_binary_edgelist = pd.read_csv(basepath+'processed_data/binary_skill_occupation_bipartite_serrano_filtered.csv')
bipartite_binary_edgelist = bipartite_binary_edgelist[['occ_8_dig','element_ID']]


print(bipartite_binary_edgelist.head())
print(bipartite_binary_edgelist.describe)

    occ_8_dig element_ID
0  11-1011.00    2.A.1.a
1  11-1011.00    2.A.1.b
2  11-1011.00    2.A.1.c
3  11-1011.00    2.A.1.d
4  11-1011.00    2.A.1.e
<bound method NDFrame.describe of         occ_8_dig element_ID
0      11-1011.00    2.A.1.a
1      11-1011.00    2.A.1.b
2      11-1011.00    2.A.1.c
3      11-1011.00    2.A.1.d
4      11-1011.00    2.A.1.e
...           ...        ...
33860  53-7121.00  1.A.4.a.6
33861  53-7121.00  1.A.4.b.1
33862  53-7121.00  1.A.4.b.2
33863  53-7121.00    2.C.2.a
33864  53-7121.00    2.C.3.e

[33865 rows x 2 columns]>


In [13]:
bipartite_binary_edgelist.loc[bipartite_binary_edgelist.occ_8_dig == "13-2099.01"]. \
    merge(skills_names, on = "element_ID").sort_values(by = "element_title").head()
# print(bipartite_binary_edgelist)

,occ_8_dig,element_ID,element_title,type
6,13-2099.01,2.A.2.b,Active Learning,skill
1,13-2099.01,2.A.1.b,Active Listening,skill
38,13-2099.01,2.C.1.a,Administration and Management,knowledge
30,13-2099.01,1.A.1.b.7,Category Flexibility,ability
14,13-2099.01,2.B.2.i,Complex Problem Solving,skill


In [14]:
print("Occupations that have both Math Skill and Programming")
print(bipartite_binary_edgelist.loc[bipartite_binary_edgelist.element_ID == "2.A.1.e"].rename(columns = {"element_ID":"Math"}). \
    merge(bipartite_binary_edgelist.loc[bipartite_binary_edgelist.element_ID == "2.B.3.e"], how = "inner"). \
      merge(occ_names, on = "occ_8_dig")[['Math','element_ID','Title']].head())

print("Occupations that have Math Skill but not Programming")
temp = bipartite_binary_edgelist.loc[bipartite_binary_edgelist.element_ID == "2.A.1.e"].rename(columns = {"element_ID":"Math"}). \
    merge(bipartite_binary_edgelist.loc[bipartite_binary_edgelist.element_ID == "2.B.3.e"], how = "left"). \
      merge(occ_names, on = "occ_8_dig")[['Math','element_ID','Title']]
print(temp[temp['element_ID'].isnull()].sample(50))

print("Occupations that have Programming but not Math Skills")
temp = bipartite_binary_edgelist.loc[bipartite_binary_edgelist.element_ID == "2.A.1.e"].rename(columns = {"element_ID":"Math"}). \
    merge(bipartite_binary_edgelist.loc[bipartite_binary_edgelist.element_ID == "2.B.3.e"], how = "right"). \
     merge(occ_names, on = "occ_8_dig")[['Math','element_ID','Title']]
print(temp[temp['Math'].isnull()].head())

#17-1011.00: Architects; 11-1011.00: CEO; 25-2031.00: secondary school teacher;
# 19-2012.00: Physist; 29-1041.00: optometrist
print(bipartite_binary_edgelist.loc[(bipartite_binary_edgelist.element_ID == "2.A.1.e") & \
                                    (bipartite_binary_edgelist.occ_8_dig == "25-2031.00")].rename(columns = {"element_ID":"Math"}). \
    merge(bipartite_binary_edgelist.loc[(bipartite_binary_edgelist.element_ID == "2.B.3.e") & \
                                        (bipartite_binary_edgelist.occ_8_dig == "25-2031.00")], how = "left"). \
     merge(occ_names, on = "occ_8_dig")[['Math','element_ID','Title']].head())



Occupations that have both Math Skill and Programming
      Math element_ID                     Title
0  2.A.1.e    2.B.3.e             Statisticians
1  2.A.1.e    2.B.3.e  Mathematical Technicians
2  2.A.1.e    2.B.3.e       Mapping Technicians
3  2.A.1.e    2.B.3.e                Biologists
4  2.A.1.e    2.B.3.e                Physicists
Occupations that have Math Skill but not Programming
        Math element_ID                                              Title
99   2.A.1.e        NaN                           Food Science Technicians
3    2.A.1.e        NaN                                     Sales Managers
76   2.A.1.e        NaN                                  Animal Scientists
178  2.A.1.e        NaN  Grinding, Lapping, Polishing, and Buffing Mach...
142  2.A.1.e        NaN           Sales Agents, Securities and Commodities
15   2.A.1.e        NaN        Purchasing Agents and Buyers, Farm Products
162  2.A.1.e        NaN                                       Order Clerks
188  

## Transforming Edgelist to Bipartite Network

In [17]:
# skill_occ_B = from_pandas_dataframe(bipartite_binary_edgelist)

skill_occ_B = nx.Graph()
skill_occ_B.add_nodes_from(bipartite_binary_edgelist['occ_8_dig'], bipartite=0)
skill_occ_B.add_nodes_from(bipartite_binary_edgelist['element_ID'], bipartite=1) # *** Remember that Skills are at dim 1 ***
skill_occ_B.add_edges_from(
    [(row['occ_8_dig'], row['element_ID']) for idx, row in bipartite_binary_edgelist.iterrows()])

# skill_occ_B = nx.from_pandas_edgelist(bipartite_binary_edgelist.rename(columns = {'occ_8_dig':"source", 'element_ID':"target"}), \
#                                       create_using=nx.DiGraph())

print(list(skill_occ_B.nodes)[0:10])
print(list(skill_occ_B.edges)[0:10])
print(len(list(skill_occ_B.edges)))

['11-1011.00', '11-1011.03', '11-1021.00', '11-2011.00', '11-2021.00', '11-2022.00', '11-2031.00', '11-3011.00', '11-3021.00', '11-3031.01']
[('11-1011.00', '2.A.1.a'), ('11-1011.00', '2.A.1.b'), ('11-1011.00', '2.A.1.c'), ('11-1011.00', '2.A.1.d'), ('11-1011.00', '2.A.1.e'), ('11-1011.00', '2.A.2.a'), ('11-1011.00', '2.A.2.b'), ('11-1011.00', '2.A.2.c'), ('11-1011.00', '2.A.2.d'), ('11-1011.00', '2.B.1.a')]
33865


# Extracting Backbone

## Source Code for Jo et al 2020

In [ ]:
## The Function Code from https://github.com/wooseongjo/hier-backbone/blob/master/hier_backbone/core.py

"""  
** Core library to extract hierarchical skill network. 
** Version: 0.8
** Date: April. 16. 2018
** Python version >= 3.x || >= 2.6

"""
from __future__ import print_function
import networkx as nx
import itertools, math, pickle, sys, re 


def write_edges(f, G, **kwargs):
    """
    Input: 
        1. file object: f
        2. graph object: G
    Output: 
        None
    
    Write edges (u, v) (direction: u ->v) with hierarchical measure alpha
    """
    try:
        weight_col = kwargs['weight']
    except:
        weight_col = 'weight'

    for u, v in G.edges_iter():
        f.write("%s\t%s\t%f\n" % (str(u), str(v), float(G[u][v][weight_col])) )

    f.close()

def parsimony_network(G):
    G_parsi = G.copy()

    n_list = list(G_parsi.nodes())
    direct_path_list = []

    for node in n_list:
        nn_set = set(G_parsi.successors(node))
        bfs_reachable_nodes = set()

        for nn in list(nn_set):
            bfs_dic = nx.bfs_successors(G_parsi, nn)
            bfs_reachable_nodes = bfs_reachable_nodes | set( [n for s, nn in bfs_dic for n in nn ] )

        cur_direct_path = bfs_reachable_nodes & nn_set

        for nn in list(cur_direct_path):
            direct_path_list.append((node, nn))
    
    print("Remove %d direct paths in G" % len(direct_path_list))

    G_parsi.remove_edges_from(direct_path_list)

    return G_parsi


def projection_from_list(tag_deg_list, tag_co_list, item_size, **kwargs):

    try:
        z_score_thres = float(kwargs['zscore'])
    except:
        z_score_thres = 2.0

    nofnode_pair_axis = int(item_size)
    tag_deg_dic = {}
    cp = {}
    edge_dic_filtered = {}

    for tag, deg in tag_deg_list:
        tag_deg_dic[tag] = int(deg)

    totlen = len(tag_co_list)
    thres_percent = 1
    
    for tag_node, deg in tag_deg_list:
        edge_dic_filtered[tag_node] = []
    
    for idx, (u, v, co_occur) in enumerate(tag_co_list):
        len_v_nn = float(tag_deg_dic[v])
        len_u_nn = float(tag_deg_dic[u])
        common_nn = float(co_occur)

        average = len_v_nn * len_u_nn / nofnode_pair_axis
        deviation = math.sqrt(len_v_nn * len_u_nn * (nofnode_pair_axis - len_v_nn) * (nofnode_pair_axis - len_u_nn) \
                / (nofnode_pair_axis * nofnode_pair_axis * (nofnode_pair_axis-1) )  )

        if ((common_nn - average)/deviation) >= z_score_thres:
            cp[(u, v)] = common_nn / len_v_nn
            cp[(v, u)] = common_nn / len_u_nn
            edge_dic_filtered[u].append(v)
            edge_dic_filtered[v].append(u)
        else:
            #print(u, v, (common_nn - average)/deviation) 
            pass

        if( (idx / float(totlen)*100.) > thres_percent):
            print("Progress: %d%%" % (int)(idx / float(totlen) * 100.0), end="\r")
            thres_percent += 1

    print("= Finish removing noisy-like edges & calculating conditional probability.")

    return edge_dic_filtered, cp



def projection_from_con(con, **kwargs):
    """
    Input: 
        1. list of connections [ (u, v), (w, x), ... , (k, t)]
        2. keyword arguemnt: axis= 0 or 1
           the list "con" has two dimension for its elements.
           Axis determines the dimension of node layer which 
           the bipartite network will projected onto.

           For example, 
               with the keyword of axis = 1, 
               an projection network with nodes in second dimension of 
               connection list is generated.
         
    
    Output: 
        1. dictionary of edges {v: [u, w, x, ...], w: [v, a, c, ...] }
        2. dictionary of conditional probability
            {(u,v): value, (v,u):value, (x,y):value, ... }
            Note: conditional prob. is asymmetric for commutation of two nodes.
            cp[(u,v)] != cp[(v, u)]

    """
    if kwargs["axis"] == 0:
        axis = 0
        pair_axis = 1
    elif kwargs["axis"] == 1:
        axis = 1
        pair_axis = 0
    else:
        print("axis: 0 or 1")
    
    try:
        z_score_thres = float(kwargs['zscore'])
    except:
        z_score_thres = 2.0
        
    
    print("-Make a dictionary")
    ## make a dic
    proj_dic = {}
    for elem in con:
        if elem[pair_axis] in proj_dic.keys():
            proj_dic[elem[pair_axis]].append(elem[axis])
        else:
            proj_dic[elem[pair_axis]] = [elem[axis]]
    
    print("-Ready to project the network")
    print("-Read %d nodes in the pair-axis layer" % len(proj_dic.keys()))
    print("-Projecting the biparitite network")
    #make edge list
    
    edge_list = {}
    
    thres_percent = 1
    totlen = len(proj_dic.keys())
    cnt = 0
    
    for key in proj_dic.keys():
        for node in proj_dic[key]:
            if node in edge_list.keys():
                edge_list[node] = edge_list[node].union(proj_dic[key])
            else:
                edge_list[node] = set(proj_dic[key])
        
        cnt += 1
        if( (cnt / float(totlen)*100.) > thres_percent):
            print("Progress: %d%%" % (int)(cnt / float(totlen) * 100.0), end="\r")
            thres_percent += 1
    print("=Complete to make projection network")
    
    for node in edge_list.keys():
        edge_list[node] = list(edge_list[node] - set([node]))
            
    nofnode_pair_axis = len(proj_dic.keys())
    del proj_dic
    
    print("-Make a dictionary")
    proj_dic2 = {}
    for elem in con:
        if elem[axis] in proj_dic2.keys():
            proj_dic2[elem[axis]].append(elem[pair_axis])
        else:
            proj_dic2[elem[axis]] = [elem[pair_axis]]
    
    for key in proj_dic2.keys():
        proj_dic2[key] = set(proj_dic2[key])
        
    deg_dic = {}
    cp= {}
    edge_dic_filtered = {}
    
    for u in proj_dic2.keys():
        deg_dic[u] = 0
        edge_dic_filtered[u] = []
        
    print("-Ready to remove random edges (z-score thres: %f)" % z_score_thres)
    print("-Read %d nodes in the axis layer" % len(proj_dic2.keys()))
    print("-Calculating z-score and conditional prob. for all edges in the projection network")   
    thres_percent = 1
    totlen = len(edge_list.keys())
    cnt = 0        
    
    z_scores = []
    for u in edge_list.keys():
        for v in edge_list[u]:
            len_v_nn = float(len(proj_dic2[v]))
            len_u_nn = float(len(proj_dic2[u]))
            common_nn = float(len(proj_dic2[v] & proj_dic2[u]))
        
            #print(u, v, len_v_nn, len_u_nn, nofnode_pair_axis)
            average = len_v_nn * len_u_nn / nofnode_pair_axis
            deviation = math.sqrt(len_v_nn * len_u_nn * (nofnode_pair_axis - len_v_nn) * (nofnode_pair_axis - len_u_nn) \
                              / (nofnode_pair_axis * nofnode_pair_axis * (nofnode_pair_axis-1) )  )

            z_scores.append(((common_nn - average)/deviation))
            if deviation == 0 or ((common_nn - average)/deviation) >= z_score_thres:
                cp[(u, v)] = common_nn / len_v_nn
                cp[(v, u)] = common_nn / len_u_nn
                edge_dic_filtered[u].append(v)
                edge_dic_filtered[v].append(u)
            else:
                #print(u, v, (common_nn - average)/deviation) 
                pass
        
        cnt += 1
        
        if( (cnt / float(totlen)*100.) > thres_percent):
            print("Progress: %d%%" % (int)(cnt / float(totlen) * 100.0), end="\r")
            thres_percent += 1
            
    print("=Complete to calculate conditional probability and remove random edges")
    
    print("Visualizing Distribution of Z-scores")
#     plt.hist(z_scores, 50)
    return edge_dic_filtered, cp
    

def extract_hierarchical_subgraph_from_edges(edge_dic, cp, thres, **kwargs):
    """
    Input: 
        1. edge_dic: dictionary of edges such as {u: [u, w, x, ...] }
        2. cp: dictionary of conditional probability obtained from the function
               of projection_from_con().
        3. thres: threshold value of alpha. 
                When you input 0 to thres, it generate the whole projection 
                network.

    Output:
        Networkx DiGraph for hierarchical subgraph 
    """
    max_k = max(list(len(v) for v in edge_dic.values()))
    
    thres_percent = 1
    totlen = len(edge_dic.keys())
    cnt = 0   

    if not 'output' in kwargs.keys():
        kwargs['output'] = "graph"
    
    if kwargs['output'] == "graph":
        print("-Option: return graph object")
        print("-Calculate the hierarchy measure, alpha")
        edges_bunch = []

        for u in edge_dic.keys():
            for v in edge_dic[u]:
                min_deg = float(min([len(edge_dic[u]), len(edge_dic[v])]))
                asym = abs(cp[(u,v)] - cp[(v, u)])
                alpha = round(min_deg * asym / max_k, 4)
        
                if alpha > thres:
                    if cp[(u,v)] > cp[(v, u)]: # root: u
                        edges_bunch.append([u, v, alpha])
                    elif cp[(u,v)] < cp[(v, u)]:  #root: v
                        edges_bunch.append([v, u, alpha])
                    else:
                        edges_bunch.append([u, v, alpha])
                        edges_bunch.append([v, u, alpha])

    
            cnt += 1
    
            if( (cnt / float(totlen)*100.) > thres_percent):
                print("Progress: %d%%" % (int)(cnt / float(totlen) * 100.0), end="\r")
                thres_percent += 1 
        
        print("=Complete the calculation of hierarhcy measure")
        print("-Making graph object using NetworkX")
        subG = nx.DiGraph()
        subG.add_weighted_edges_from(edges_bunch, 'alpha')
        print("=Complete!")
        
        return subG
    
    elif kwargs['output'] == 'file':
        try:
            fn = kwargs['fn']
        except KeyError:
            print("Input file name as keywords: fn=\'alpha.txt\'")
        
        
        fs = open(fn, 'w')
        fs.write("#Direction: u --> v\n")
        fs.write("#u\tv\talpha\n")

        print("-Option: write results in given file")
        print("-Calculate the hierarchy measure, alpha")
        #for u, v in edge_pair_list:
        for u in edge_dic.keys():
            for v in edge_dic[u]:
                min_deg = float(min([len(edge_dic[u]), len(edge_dic[v])]))
                asym = abs(cp[(u,v)] - cp[(v, u)])
                alpha = min_deg * asym / max_k
                 
                if alpha > thres:
                    if cp[(u,v)] > cp[(v, u)]: # root: u
                        fs.write("%s\t%s\t%f\n" % (u, v, alpha) )
                    elif cp[(u,v)] < cp[(v, u)]:  #root: v
                        fs.write("%s\t%s\t%f\n" % (v, u, alpha) )
                    else:
                        fs.write("%s\t%s\t%f\n" % (u, v, alpha) )
                        fs.write("%s\t%s\t%f\n" % (v, u, alpha) )
    
            cnt += 1
    
            if( (cnt / float(totlen)*100.) > thres_percent):
                print("Progress: %d%%" % (int)(cnt / float(totlen) * 100.0), end="\r")
                thres_percent += 1 
        
        print("Save the result to %s" % fn)
        fs.close()


def edges_file_to_graph(f):

    hbG = nx.DiGraph()
    
    cnt = 0
    thres_cnt = 100

    while True:
        line = f.readline()
        if not line: break
        if line[0] == "#": continue
        elem = re.split("\t|\n", line)
        hbG.add_edge(elem[0], elem[1], {'weight':float(elem[2])})
        cnt+=1

        if ( cnt / 100.) > thres_cnt:
            print("Read edges: %d" % cnt, end="\r")
            thres_cnt += 100

    
    return hbG


def bipartite_projection(B, target_node_list):
    """
    Input
        B: bipartite network
        target_node_list: a bunch of nodes divided to be parallelized 
        output: instance of Queue

    Output:
        returns edge list with weight 
        [[u, v, w], [u, v, w], ... ]

    Find connections from a bunch of nodes to make projection network
    It returns edge list with weight such as [u, v, weight]

    """
    r = []
    pred=B.adj
    totlen = len(target_node_list)
    cnt=0
    thres_percent = 1
    for u in target_node_list:
        unbrs = set(B[u])
        nbrs2 = set((n for nbr in unbrs for n in B[nbr])) - set([u])
        for v in nbrs2:
            vnbrs = set(pred[v])
            common = unbrs & vnbrs
            weight = len(common)
            r.append([u, v, weight])

        cnt+=1
        if( (cnt / float(totlen) * 100.) > thres_percent):
            print("Process: %d%%,  Used memory: %d" % ((int)(cnt / float(totlen) * 100.), len(r) ))
            thres_percent += 1
    
    r2 = [ [u, v, {'weight': w}] for u, v, w in r ]
    proG = nx.Graph()
    proG.add_edges_from(r2)

    return proG


def zscore(Q, Qi, Qj, Qij):
    """
    Calculate z-score from probability of co-occurance 
    between two any skills in the given biparaite network.
    
    Inputs:
        Q  : The number of users 
        Qi : The number of users who connect to i-th skill 
        Qj : The number of users who connect to j-th skill.
        Qij: The number of users who connect to both of 
            the i-th skill and the j-th skill.
    
    Return:
        It returns z-score value.
    """

    average = float(Qj)*float(Qi)/float(Q) 
    deviation = math.sqrt(Qi*Qj*(Q-Qi)*(Q-Qj) / float(Q*Q*(Q-1)) ) 

    return (Qij - average) / deviation


def find_communities_from_hierarchy(G, **kwargs):
    newman_ziff_dic = {}
    comm_dic = {}

    #### Check the prev_hier
    if "prev_hier" in kwargs.keys() and "new_edge" in kwargs.keys():
        prev_hier = kwargs["prev_hier"]
        new_edge = kwargs["new_edge"]
        
        newman_ziff_dic = prev_hier.copy()
        
        ## new_edge: (u, v)
        ## Direction: u -> v
        ## if u node is incoming in the graph G, find its root
        if not u in prev_hier.keys() and not v in prev_hier.keys():
           newman_ziff_dic[u] = -1 
           newman_ziff_dic[v] = u 
        
        elif u in prev_hier.keys() and not v in prev_hier.keys():
           newman_ziff_dic[v] = u 
        elif not u in prev_hier.keys() and v in prev_hier.keys():
            newman_ziff_dic[u] = -1 
            
            ## update v's parent
            max_weight = 0
            for parent in G.in_edges(v):
                if G[parent][v]['weight'] > max_weight:
                    parent_node = parent
            newman_ziff_dic[v] = parent_node
        
        else:
            ## update u's parent
            max_weight = 0
            for parent in G.in_edges(u):
                if G[parent][u]['weight'] > max_weight:
                    parent_node = parent
            newman_ziff_dic[u] = parent_node
        
            ## update v's parent
            max_weight = 0
            for parent in G.in_edges(v):
                if G[parent][v]['weight'] > max_weight:
                    parent_node = parent
            newman_ziff_dic[v] = parent_node

        ### find communities
        for u in newman_ziff_dic.keys():
            comm_dic[u] = find_root_hierarchy_from_newman_ziff(newman_ziff_dic, u)                   

    else:
        ## find the most probable parent node
        for u in G.nodes_iter():
            if G.in_degree(u) == 0:
                newman_ziff_dic[u] = -1
            else:
                max_weight = 0
                for parent, cur_u in G.in_edges(u):
                    if G[parent][cur_u]['weight'] > max_weight:
                        max_weight = G[parent][cur_u]['weight'] 
                        parent_node = parent
                newman_ziff_dic[u] = parent_node
        
        ### find communities
        for u in newman_ziff_dic.keys():
            comm_dic[u] = find_root_hierarchy_from_newman_ziff(newman_ziff_dic, u)                   

    return comm_dic, newman_ziff_dic


## Step-by-Step of Jo et al (2020) Algorithm

### Step 1: One-side Projection - for us skills in dimension 1

In [25]:
z_thresh = 4.75
skill_side_prj, skill_side_cp = projection_from_con(skill_occ_B.edges, axis = 1, zscore = z_thresh)

print(list(skill_side_prj.items())[0])
print(len(list(skill_side_prj.items())))
print(list(skill_side_cp.items())[0])
print(len(list(skill_side_cp.items())))
    
## The outcome is called Pruned graph in the paper.
    

-Make a dictionary
-Ready to project the network
-Read 968 nodes in the pair-axis layer
-Projecting the biparitite network
=Complete to make projection network
-Make a dictionary
-Ready to remove random edges (z-score thres: 4.750000)
-Read 120 nodes in the axis layer
-Calculating z-score and conditional prob. for all edges in the projection network
=Complete to calculate conditional probability and remove random edges
Visualizing Distribution of Z-scores
('2.A.1.a', ['2.C.4.d', '1.A.1.b.2', '2.C.9.b', '2.C.5.b', '2.A.1.e', '2.B.5.b', '2.B.3.a', '2.A.1.b', '1.A.1.b.5', '1.A.1.b.6', '2.B.5.a', '2.B.4.h', '2.C.7.a', '2.B.1.f', '2.C.4.f', '1.A.1.b.3', '2.C.1.a', '2.C.4.b', '2.B.5.d', '1.A.1.e.2', '2.C.7.e', '2.A.2.a', '1.A.1.b.4', '1.A.1.c.2', '2.A.2.d', '2.C.8.b', '1.A.1.b.7', '2.C.1.b', '2.C.1.e', '1.A.4.b.5', '2.B.1.d', '2.A.1.f', '2.C.4.a', '1.A.4.b.4', '2.A.2.b', '2.B.4.e', '1.A.1.c.1', '2.A.1.c', '2.B.1.b', '1.A.4.a.1', '2.B.2.i', '2.A.2.c', '1.A.1.e.1', '1.A.1.a.3', '2.C.4.g', '1.A

### Step 2: Produce a Hierarchical Graph

In [88]:
a_thresh = 0.05
hierarchical_skill_G = extract_hierarchical_subgraph_from_edges(skill_side_prj, skill_side_cp, thres = a_thresh)

print("previous number of edges:" + str(len(list(skill_side_cp.items()))))

print("number of nodes left:" + str(len(hierarchical_skill_G.nodes)))
print(list(hierarchical_skill_G.nodes)[:5])
print("number of edges left:" + str(len(hierarchical_skill_G.edges)))
print(list(hierarchical_skill_G.edges)[:5])
print(list(nx.get_edge_attributes(hierarchical_skill_G,'alpha').items())[:5])


-Option: return graph object
-Calculate the hierarchy measure, alpha
=Complete the calculation of hierarhcy measure
-Making graph object using NetworkX
=Complete!
previous number of edges:4322
number of nodes left:115
['2.A.1.a', '2.C.4.d', '1.A.1.b.2', '2.C.9.b', '2.C.5.b']
number of edges left:1796
[('2.A.1.a', '2.C.4.d'), ('2.A.1.a', '1.A.1.b.2'), ('2.A.1.a', '2.C.9.b'), ('2.A.1.a', '2.C.5.b'), ('2.A.1.a', '2.A.1.e')]
[(('2.A.1.a', '2.C.4.d'), 0.2954), (('2.A.1.a', '1.A.1.b.2'), 0.2191), (('2.A.1.a', '2.C.9.b'), 0.4186), (('2.A.1.a', '2.C.5.b'), 0.4862), (('2.A.1.a', '2.A.1.e'), 0.3641)]


### Quick Dirty Visualization

In [ ]:
pos = nx.nx_agraph.graphviz_layout(hierarchical_skill_G, prog='dot')
nx.draw(hierarchical_skill_G, node_size=50, pos=pos, width=list(nx.get_edge_attributes(hierarchical_skill_G,'alpha').values()))


#### Saving the graph without additional features for possible external use

In [ ]:
# Save hierarchical skill network edgelist
hierarchical_skill_edgelist = nx.to_pandas_edgelist(hierarchical_skill_G)
hierarchical_skill_edgelist = hierarchical_skill_edgelist. \
    merge(skills_names[["element_title","element_ID"]].rename(columns = {"element_title":"source_title", "element_ID":"source"}), on = "source"). \
    merge(skills_names[["element_title","element_ID"]].rename(columns = {"element_title":"target_title", "element_ID":"target"}), on = "target")

print(hierarchical_skill_edgelist)
hierarchical_skill_edgelist.\
    to_csv(basepath+'output_data/' \
           'hierarchical_skill_network_edgelist_z-thres_'+str(z_thresh)+'_a-thres_'+str(a_thresh)+'.csv', index=False)

         source     target   alpha             source_title  \
0       2.A.1.a    2.C.4.d  0.2954    Reading Comprehension   
1     1.A.1.b.5    2.C.4.d  0.2778      Inductive Reasoning   
2     1.A.1.b.4    2.C.4.d  0.2818      Deductive Reasoning   
3     1.A.4.b.5    2.C.4.d  0.2151           Speech Clarity   
4       2.C.4.a    2.C.4.d  0.2540    Mathematics Knowledge   
...         ...        ...     ...                      ...   
1791  1.A.3.c.3  1.A.3.c.2  0.0602  Gross Body Coordination   
1792  1.A.3.a.3  1.A.3.c.2  0.0603         Dynamic Strength   
1793  1.A.2.b.3  1.A.4.a.4  0.0608     Response Orientation   
1794  1.A.1.f.1  1.A.4.a.4  0.0565      Spatial Orientation   
1795  1.A.1.g.1  1.A.2.a.3  0.0578      Selective Attention   

             target_title  
0                 Biology  
1                 Biology  
2                 Biology  
3                 Biology  
4                 Biology  
...                   ...  
1791  Dynamic Flexibility  
1792  Dynamic Flexi

### Step 3: Creating a Parsimonuous Grapgh

In [98]:
parsi_G = parsimony_network(hierarchical_skill_G)

print("number of nodes left:" + str(len(parsi_G.nodes)))
print("number of edges left:" + str(len(parsi_G.edges)))

print("\nList of Missing Skills:")
print(skills_names[~skills_names.element_ID.isin(list(e[0] for e in parsi_G.nodes(data=True)))])


Remove 1402 direct paths in G
number of nodes left:115
number of edges left:394

List of Missing Skills:
       element_ID               element_title       type
40        2.B.3.d                Installation      skill
67824   1.A.3.a.2          Explosive Strength    ability
67858   1.A.4.b.3          Sound Localization    ability
168446    2.C.2.b             Food Production  knowledge
168488    2.C.8.a  Public Safety and Security  knowledge


In [ ]:
nx.draw(parsi_G, node_size=50, pos=nx.nx_agraph.graphviz_layout(parsi_G, prog='dot'))


### Optional Analysis:

#### Normalizing alphas

In [107]:
##### Changing the direction of Edges so that arrow direction show dependency (child -> parent as opposed to parent -> child)
for e, f, g in parsi_G.edges(data = True):
    nx.set_edge_attributes(parsi_G, {(e, f): {"alpha_sqrt": np.sqrt(g['alpha'])}})

for e, f, g in hierarchical_skill_G.edges(data = True):
    nx.set_edge_attributes(hierarchical_skill_G, {(e, f): {"alpha_sqrt": np.sqrt(g['alpha'])}})

    
print(list(parsi_G.edges(data = True))[0:10])


[('2.A.1.a', '1.A.1.b.5', {'alpha': 0.0879, 'alpha_sqrt': 0.29647934160747186}), ('2.A.1.a', '1.A.1.b.4', {'alpha': 0.0588, 'alpha_sqrt': 0.24248711305964282}), ('2.A.1.a', '2.C.4.a', {'alpha': 0.0532, 'alpha_sqrt': 0.2306512518934159}), ('2.A.1.a', '2.A.2.b', {'alpha': 0.0908, 'alpha_sqrt': 0.3013303834663873}), ('2.A.1.a', '2.B.4.e', {'alpha': 0.109, 'alpha_sqrt': 0.3301514803843836}), ('2.A.1.a', '2.B.1.b', {'alpha': 0.0615, 'alpha_sqrt': 0.24799193535274489}), ('2.A.1.a', '1.A.1.a.4', {'alpha': 0.0969, 'alpha_sqrt': 0.3112876483254676}), ('2.A.1.a', '2.A.1.d', {'alpha': 0.066, 'alpha_sqrt': 0.2569046515733026}), ('2.A.1.a', '2.C.6', {'alpha': 0.0585, 'alpha_sqrt': 0.24186773244895649}), ('2.C.4.d', '2.C.5.a', {'alpha': 0.0903, 'alpha_sqrt': 0.3004995840263344})]


### Creating Z-layer Attribute based on Local Reaching Centrality - Used in Gephi

In [ ]:
for s in tqdm(parsi_G.nodes):
    nx.set_node_attributes(parsi_G, {s: {"LRC": nx.local_reaching_centrality(parsi_G, s, weight = "alpha", normalized = True)}})
    nx.set_node_attributes(parsi_G, {s: {"LRC_norm": math.floor(100 * nx.local_reaching_centrality(parsi_G, s, weight = "alpha", normalized = True))}})
    nx.set_node_attributes(parsi_G, {s: {"LRC_modif": (250 * np.log10(1 + nx.local_reaching_centrality(parsi_G, s, weight = "alpha", normalized = True)))}})
    nx.set_node_attributes(parsi_G, {s: {"LRC_non_parsi_modif": (250 * np.log10(1 + nx.local_reaching_centrality(hierarchical_skill_G, s, weight = "alpha", normalized = True)))}})
    nx.set_node_attributes(hierarchical_skill_G, {s: {"LRC_non_parsi": nx.local_reaching_centrality(hierarchical_skill_G, s, weight = "alpha", normalized = True)}})

    
for s in tqdm(hierarchical_skill_G.nodes):
    nx.set_node_attributes(hierarchical_skill_G, {s: {"LRC": nx.local_reaching_centrality(hierarchical_skill_G, s, weight = "alpha", normalized = True)}})
    nx.set_node_attributes(hierarchical_skill_G, {s: {"LRC_norm": math.floor(100 * nx.local_reaching_centrality(hierarchical_skill_G, s, weight = "alpha", normalized = True))}})
    nx.set_node_attributes(hierarchical_skill_G, {s: {"LRC_modif": (250 * np.log10(1 + nx.local_reaching_centrality(hierarchical_skill_G, s, weight = "alpha", normalized = True)))}})



z_vals = nx.get_node_attributes(parsi_G,"LRC")

# Save Local Reaching Centrality measures for both networks
pd.DataFrame.from_dict(nx.get_node_attributes(hierarchical_skill_G,"LRC"), orient = "index")\
    .reset_index()\
    .rename(columns = {'index': 'element_ID', 0:'LRC'})\
    .to_csv(basepath+'output_data/' \
           'skill_dependency_network_LRC_measures.csv', index=False)

pd.DataFrame.from_dict(nx.get_node_attributes(parsi_G,"LRC"), orient = "index")\
    .reset_index()\
    .rename(columns = {'index': 'element_ID', 0:'LRC'})\
    .to_csv(basepath+'output_data/' \
           'parsimonious_skill_dependency_network_LRC_measures.csv', index=False)


print(nx.get_node_attributes(parsi_G,"LRC_modif"))

100%|████████████████████████████████████████| 115/115 [00:00<00:00, 658.60it/s]

{'2.A.1.a': 62.03969274786624, '2.C.4.d': 0.9758932702239759, '1.A.1.b.2': 37.382999437546715, '2.C.9.b': 12.113167885508942, '2.C.5.b': 0.0, '2.A.1.e': 23.101712411748764, '2.B.5.b': 1.5017992291560522, '2.B.3.a': 4.775867247294729, '1.A.1.b.5': 50.12684630972392, '2.B.5.a': 35.24109622027696, '2.B.4.h': 30.847179650801955, '2.B.1.f': 34.55691214955679, '2.C.4.f': 6.793718948105963, '2.C.1.a': 37.889212212256616, '2.C.4.b': 0.0, '2.B.5.d': 18.36766486826223, '1.A.1.e.2': 44.151835766129615, '2.C.7.e': 2.0496746004128417, '1.A.1.b.4': 53.078812544092486, '1.A.1.c.2': 22.72269080123369, '2.C.8.b': 17.028019525341744, '2.C.1.b': 36.65242155349975, '1.A.4.b.5': 35.096557407766696, '2.B.1.d': 18.973092664007343, '2.A.1.f': 0.7292365510458529, '2.C.4.a': 46.608280666042326, '2.A.2.b': 50.070444303274876, '2.B.4.e': 53.41756068789664, '1.A.1.c.1': 23.44851277990787, '2.A.1.c': 52.204115766481195, '2.B.1.b': 41.47240168896855, '1.A.4.a.1': 70.76563404836213, '2.B.2.i': 50.665586817828796, '2.

### Adding Node Feature as Ancestors of a few Skills

In [ ]:
# Load skill dependency data computed using Baldwin (2010) method
skill_dependency_df = pd.read_csv(basepath+'processed_data/skill_dependency_data_baldwin_method.csv')

print(skill_dependency_df.head())


          ID  Prog_cum_dep  Repr_cum_dep  Nego_cum_dep  Prog_norm_dep  \
0  1.A.1.a.1      0.135337      0.000100      0.040161       0.135337   
1  1.A.1.a.2      0.133336      0.000055      0.038556       0.133336   
2  1.A.1.a.3      0.136986      0.000100      0.041227       0.136986   
3  1.A.1.a.4      0.124314      0.000024      0.035521       0.124314   
4  1.A.1.b.1      0.156736      0.000000      0.035617       0.156736   

   Repr_norm_dep  Nego_norm_dep  
0       0.000324       0.589696  
1       0.000177       0.566132  
2       0.000325       0.605344  
3       0.000076       0.521559  
4       0.000000       0.522981  


### Final Step: Adding Other Attribures and Save Graph as Gephi File

In [ ]:
#### Saving Node Names, Education, and Clusters as dictionary files
skill_names_and_clusters = skills_names[["element_ID", "element_title", "type"]] \
    .merge(skill_clusters[["element_ID", "skill_Cluster", "gen_related"]], on = "element_ID") \
    .merge(pd.DataFrame([["General",{'color':{'r': 230, 'g': 7, 'b': 42, 'a': 1}}], ["Intermediate",{'color':{'r': 190, 'g': 190, 'b': 190, 'a': 1}}], ["Specific",{'color':{'r': 66, 'g':79, 'b': 164, 'a': 1}}]], columns = ["skill_Cluster","viz"]), on = "skill_Cluster") \
    .merge(skill_educ, on = 'element_ID') \
    .merge(skill_educ_2005limited, on = 'element_ID') \
    .merge(skill_wages, on = 'element_ID')

print(skill_names_and_clusters.head())

skill_names_and_clusters = skill_names_and_clusters.set_index('element_ID').T.to_dict('dict')

print(list(skill_names_and_clusters.items())[:5])

#### Adding these attributes to the graph
nx.set_node_attributes(hierarchical_skill_G, skill_names_and_clusters)
nx.set_node_attributes(parsi_G, skill_names_and_clusters)

# print(skill_names_and_clusters.head())
print(list(parsi_G.nodes)[:5])
print(list(parsi_G.edges)[:5])
print(list(nx.get_edge_attributes(parsi_G,'alpha').items())[:5])
print(list(nx.get_node_attributes(parsi_G,'[z]').items())[:5])



  element_ID          element_title   type skill_Cluster gen_related  \
0    2.A.1.a  Reading Comprehension  skill       General     General   
1    2.A.1.b       Active Listening  skill       General     General   
2    2.A.1.c                Writing  skill       General     General   
3    2.A.1.d               Speaking  skill       General     General   
4    2.A.2.a      Critical Thinking  skill       General     General   

                                              viz  Imp_weight_Avg_Edu  \
0  {'color': {'r': 230, 'g': 7, 'b': 42, 'a': 1}}               5.007   
1  {'color': {'r': 230, 'g': 7, 'b': 42, 'a': 1}}               4.914   
2  {'color': {'r': 230, 'g': 7, 'b': 42, 'a': 1}}               5.079   
3  {'color': {'r': 230, 'g': 7, 'b': 42, 'a': 1}}               4.928   
4  {'color': {'r': 230, 'g': 7, 'b': 42, 'a': 1}}               4.934   

   Lvl_weight_Avg_Edu  Lvl_weight_2005limited_Edu  log_wage  
0               5.198                       5.463     4.796  
1   

### Define Edge Colors

In [121]:
color_mapping = {
    'General': {'r': 230, 'g': 7, 'b': 42, 'a': 1},
    'Intermediate': {'r': 89, 'g': 89, 'b': 89, 'a': 1},
    'Specific': {'r': 66, 'g': 79, 'b': 164, 'a': 1}
}

# Assign colors to edges based on the parent node's skill_cluster
for u, v in parsi_G.edges():
    parent_skill_cluster = parsi_G.nodes[u]['skill_Cluster']
    edge_color = color_mapping.get(parent_skill_cluster, {'r': 0, 'g': 0, 'b': 0, 'a': 1})
    parsi_G[u][v]['viz'] = {'color': edge_color}
    
    
for u, v in hierarchical_skill_G.edges():
    parent_skill_cluster = hierarchical_skill_G.nodes[u]['skill_Cluster']
    edge_color = color_mapping.get(parent_skill_cluster, {'r': 0, 'g': 0, 'b': 0, 'a': 1})
    hierarchical_skill_G[u][v]['viz'] = {'color': edge_color}

### Saving Files

In [ ]:
# Save networks in GEXF format for visualization in Gephi
nx.write_gexf(parsi_G, \
              basepath + 'output_data/gephi_files/' + \
              'parsimonious_skill_network_z-thres_' + str(z_thresh) + '_a-thres_' + str(a_thresh) + '.gexf')

nx.write_gexf(hierarchical_skill_G, \
              basepath + 'output_data/gephi_files/' + \
              'hierarchical_skill_network_z-thres_' + str(z_thresh) + '_a-thres_' + str(a_thresh) + '.gexf')

In [ ]:
parsi_G_edgelist = nx.to_pandas_edgelist(parsi_G)

## Saving the Parsimonious Network Edgelist
parsi_G_edgelist.\
    to_csv(basepath+'output_data/' \
           'parsimonious_skill_dependency_edgelist.csv', index=False)

print(parsi_G_edgelist.head())

parsi_G_nodelist = skills_names[["element_ID", "element_title", "type"]] \
    .merge(skill_clusters[["element_ID", "skill_Cluster"]], on = "element_ID") \
    .merge(pd.DataFrame([["General",{'color':{'r': 255, 'g': 0, 'b': 0, 'a': 0}}], ["Common",{'color':{'r': 0, 'g': 255, 'b': 0, 'a': 0}}], ["Specific",{'color':{'r': 0, 'g': 0, 'b': 255, 'a': 0}}]], columns = ["skill_Cluster","viz"]), on = "skill_Cluster") \
    .merge(pd.DataFrame([["General",'r'], ["Common",'g'], ["Specific",'b']], columns = ["skill_Cluster","simple_color"]), on = "skill_Cluster") \
    .merge(skill_educ, on = 'element_ID') \
    .merge(skill_nested, on = 'element_ID').rename(columns = {'element_ID':'ID', "element_title":"Title", "c_i":'specialty'})

print(parsi_G_nodelist.head())

## Saving the Parsimonious Network Nodelist
parsi_G_nodelist.\
    to_csv(basepath+'output_data/' \
           'parsimonious_skill_dependency_nodelist.csv', index=False)

    source     target   alpha
0  2.A.1.a  1.A.1.a.4  0.0969
1  2.A.1.a  1.A.1.b.5  0.0879
2  2.A.1.a  1.A.1.b.4  0.0588
3  2.A.1.a    2.C.4.a  0.0532
4  2.A.1.a    2.A.1.d  0.0660
        ID                  Title   type skill_Cluster  \
0  2.A.1.a  Reading Comprehension  skill       General   
1  2.A.1.b       Active Listening  skill       General   
2  2.A.1.c                Writing  skill       General   
3  2.A.1.d               Speaking  skill       General   
4  2.A.2.a      Critical Thinking  skill       General   

                                             viz simple_color  \
0  {'color': {'r': 255, 'g': 0, 'b': 0, 'a': 0}}            r   
1  {'color': {'r': 255, 'g': 0, 'b': 0, 'a': 0}}            r   
2  {'color': {'r': 255, 'g': 0, 'b': 0, 'a': 0}}            r   
3  {'color': {'r': 255, 'g': 0, 'b': 0, 'a': 0}}            r   
4  {'color': {'r': 255, 'g': 0, 'b': 0, 'a': 0}}            r   

   Imp_weight_Avg_Edu  Lvl_weight_Avg_Edu  specialty  
0               5.007    

#### Saving The Edge List of Parsimonious Graph

In [ ]:
parsi_G_edgelist = nx.to_pandas_edgelist(parsi_G)

# Save parsimonious network edgelist
parsi_G_edgelist.\
    to_csv(basepath+'output_data/' \
           'parsimonious_network_edgelist_z-thres_'+str(z_thresh)+'_a-thres_'+str(a_thresh)+'.csv', index=False)